# Día 4: Abstract Base Classes (ABC)

## Descripción General

Las **Abstract Base Classes (ABC)** en Python permiten definir interfaces y contratos que las subclases deben cumplir. Son fundamentales para crear arquitecturas extensibles y garantizar que las clases derivadas implementen métodos específicos.

El módulo `abc` de Python proporciona la infraestructura para definir clases abstractas, métodos abstractos, y propiedades abstractas.

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. Crear clases base abstractas usando el módulo abc
2. Definir métodos abstractos que las subclases deben implementar
3. Usar ABC para definir interfaces y contratos
4. Aplicar ABC en diseños de software extensibles
5. Comprender cuándo usar ABC vs protocolos

## Recursos y Referencias Oficiales

### Documentación Oficial
- **[abc Module Documentation](https://docs.python.org/3/library/abc.html)**: Documentación oficial del módulo abc
- **[PEP 3119 - Abstract Base Classes](https://peps.python.org/pep-3119/)**: PEP que introduce ABC en Python

### Notas Importantes
- Todos los enlaces están actualizados a partir de 2025
- Se recomienda revisar la documentación oficial regularmente